# Unipressed

**Please visit the [project website](https://multimeric.github.io/Unipressed/) for more comprehensive documentation.**

## Introduction

Unipressed (Uniprot REST) is an API client for the protein database [Uniprot](https://www.uniprot.org/).
It provides thoroughly typed and documented code to ensure your use of the library is easy, fast, and correct!

### Example
Let's say we're interested in very long proteins that are encoded within a chloroplast, in any organism:

In [1]:
import json
from unipressed import Uniprotkb

for record in Uniprotkb.search(
    query={
        "and_": [
            {"organelle": "chloroplast"},
            {"length": (5000, "*")}
        ]
    },
    fields=["length", "gene_names"]
).each_record():
    display(record)

{'primaryAccession': 'A0A088CK67',
 'genes': [{'geneName': {'evidences': [{'evidenceCode': 'ECO:0000313',
      'source': 'EMBL',
      'id': 'AID67672.1'}],
    'value': 'ftsH'}}],
 'sequence': {'length': 5242}}

### Advantages

* Detailed type hints for autocompleting queries as you type
* Autocompletion for return fields
* Documentation for each field
* Automatic results parsing, for `json`, `tsv`, `list`, and `xml`
* Built-in pagination, so you don't have to handle any of that yourself!
* Most of the API is automatically generated, ensuring very rapid updates whenever the API changes
* Thoroughly tested, with 41 unit tests and counting!

## Usage

### Installation

If you're using poetry:
```bash
poetry add unipressed
```

Otherwise:
```bash
pip install unipressed
```

### Dataset Clients

The `unipressed` module exports a client object for each UniProt dataset:

In [2]:
# TODO: once https://github.com/Textualize/rich/issues/2485 and https://github.com/Textualize/rich/issues/2486
# are closed, we can change to using Rich's pretty printing

# This replaces the default Jupyter print with pprint, which elides deep nested objects for concision
import pprint
printer = pprint.PrettyPrinter(depth=1, indent=4)
for key in ["text/html", "text/markdown"]:
     get_ipython().display_formatter.formatters[key].for_type(object, printer.pformat)

In [3]:
from unipressed import Uniprotkb, Uniparc

With one of these clients, you can search the dataset:

In [4]:
records = Uniprotkb.search({
    "length": (5000, 6000)
}).each_record()

# Show the first record
next(records)

{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
 'primaryAccession': 'Q96RW7',
 'secondaryAccessions': ['A6NGE3', 'Q5TYR7', 'Q96DN3', 'Q96DN8', 'Q96SC3'],
 'uniProtkbId': 'HMCN1_HUMAN',
 'entryAudit': {'firstPublicDate': '2006-01-10',
  'lastAnnotationUpdateDate': '2022-08-03',
  'lastSequenceUpdateDate': '2006-01-10',
  'entryVersion': 173,
  'sequenceVersion': 2},
 'annotationScore': 5.0,
 'organism': {'scientificName': 'Homo sapiens',
  'commonName': 'Human',
  'taxonId': 9606,
  'lineage': ['Eukaryota',
   'Metazoa',
   'Chordata',
   'Craniata',
   'Vertebrata',
   'Euteleostomi',
   'Mammalia',
   'Eutheria',
   'Euarchontoglires',
   'Primates',
   'Haplorrhini',
   'Catarrhini',
   'Hominidae',
   'Homo']},
 'proteinExistence': '1: Evidence at protein level',
 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Hemicentin-1'}},
  'alternativeNames': [{'fullName': {'value': 'Fibulin-6'},
    'shortNames': [{'value': 'FIBL-6'}]}],
  'flag': 'Precursor'},
 'genes': [{'geneName': {'value': 'HMCN1'}, 'synonyms': [{'value': 'FIBL6'}]}],
 'comments': [{'texts': [{'evidences': [{'evidenceCode': 'ECO:0000250',
       'source': 'UniProtKB',
       'id': 'D3YXG0'},
      {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '29488390'}],
     'value': 'Involved in transforming growth factor beta-mediated rearrangement of the podocyte cytoskeleton which includes reduction of F-actin fibers and broadening, flattening and elongation of podocytes (PubMed:29488390). Plays a role in basement membrane organization (By similarity). May promote cleavage furrow maturation during cytokinesis in preimplantation embryos (By similarity). May play a role in the architecture of adhesive and flexible epithelial cell junctions (By similarity). May play a role during myocardial remodeling by imparting an effect on cardiac fibroblast migration (By similarity)'}],
   'commentType': 'FUNCTION'},
  {'commentType': 'INTERACTION',
   'interactions': [{'interactantOne': {'uniProtKBAccession': 'Q96RW7',
      'intActId': 'EBI-2806183'},
     'interactantTwo': {'uniProtKBAccession': 'P0C7Q2',
      'geneName': 'ARMS2',
      'intActId': 'EBI-21986906'},
     'numberOfExperiments': 4,
     'organismDiffer': False}]},
  {'commentType': 'SUBCELLULAR LOCATION',
   'note': {'texts': [{'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Has been detected in the glomerular basement membrane in one study. However, another study found expression in the glomerular mesangial matrix but not in the glomerular basement membrane. The antibody used to determine subcellular location does not distinguish between HMCN1 and HMCN2'}]},
   'subcellularLocations': [{'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Secreted, extracellular space, extracellular matrix, basement membrane',
      'id': 'SL-0025'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cytoplasm',
      'id': 'SL-0086'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cell junction',
      'id': 'SL-0038'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cleavage furrow',
      'id': 'SL-0467'}}]},
  {'commentType': 'ALTERNATIVE PRODUCTS',
   'events': ['Alternative splicing'],
   'isoforms': [{'name': {'value': '1'},
     'isoformIds': ['Q96RW7-1'],
     'isoformSequenceStatus': 'Displayed'},
    {'name': {'value': '2'},
     'isoformIds': ['Q96RW7-2'],
     'sequenceIds': ['VSP_016874'],
     'isoformSequenceStatus': 'Described'},
    {'name': {'value': '3'},
     'isoformIds': ['Q96RW7-3'],
     'sequenceIds': ['VSP_016871', 'VSP_016872', 'VSP_016873'],
     'isoformSequenceS

You can request a single record by ID:

In [5]:
Uniprotkb.fetch_one("Q96RW7")

{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
 'primaryAccession': 'Q96RW7',
 'secondaryAccessions': ['A6NGE3', 'Q5TYR7', 'Q96DN3', 'Q96DN8', 'Q96SC3'],
 'uniProtkbId': 'HMCN1_HUMAN',
 'entryAudit': {'firstPublicDate': '2006-01-10',
  'lastAnnotationUpdateDate': '2022-08-03',
  'lastSequenceUpdateDate': '2006-01-10',
  'entryVersion': 173,
  'sequenceVersion': 2},
 'annotationScore': 5.0,
 'organism': {'scientificName': 'Homo sapiens',
  'commonName': 'Human',
  'taxonId': 9606,
  'lineage': ['Eukaryota',
   'Metazoa',
   'Chordata',
   'Craniata',
   'Vertebrata',
   'Euteleostomi',
   'Mammalia',
   'Eutheria',
   'Euarchontoglires',
   'Primates',
   'Haplorrhini',
   'Catarrhini',
   'Hominidae',
   'Homo']},
 'proteinExistence': '1: Evidence at protein level',
 'proteinDescription': {'recommendedName': {'fullName': {'value': 'Hemicentin-1'}},
  'alternativeNames': [{'fullName': {'value': 'Fibulin-6'},
    'shortNames': [{'value': 'FIBL-6'}]}],
  'flag': 'Precursor'},
 'genes': [{'geneName': {'value': 'HMCN1'}, 'synonyms': [{'value': 'FIBL6'}]}],
 'comments': [{'texts': [{'evidences': [{'evidenceCode': 'ECO:0000250',
       'source': 'UniProtKB',
       'id': 'D3YXG0'},
      {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '29488390'}],
     'value': 'Involved in transforming growth factor beta-mediated rearrangement of the podocyte cytoskeleton which includes reduction of F-actin fibers and broadening, flattening and elongation of podocytes (PubMed:29488390). Plays a role in basement membrane organization (By similarity). May promote cleavage furrow maturation during cytokinesis in preimplantation embryos (By similarity). May play a role in the architecture of adhesive and flexible epithelial cell junctions (By similarity). May play a role during myocardial remodeling by imparting an effect on cardiac fibroblast migration (By similarity)'}],
   'commentType': 'FUNCTION'},
  {'commentType': 'INTERACTION',
   'interactions': [{'interactantOne': {'uniProtKBAccession': 'Q96RW7',
      'intActId': 'EBI-2806183'},
     'interactantTwo': {'uniProtKBAccession': 'P0C7Q2',
      'geneName': 'ARMS2',
      'intActId': 'EBI-21986906'},
     'numberOfExperiments': 4,
     'organismDiffer': False}]},
  {'commentType': 'SUBCELLULAR LOCATION',
   'note': {'texts': [{'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Has been detected in the glomerular basement membrane in one study. However, another study found expression in the glomerular mesangial matrix but not in the glomerular basement membrane. The antibody used to determine subcellular location does not distinguish between HMCN1 and HMCN2'}]},
   'subcellularLocations': [{'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Secreted, extracellular space, extracellular matrix, basement membrane',
      'id': 'SL-0025'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cytoplasm',
      'id': 'SL-0086'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cell junction',
      'id': 'SL-0038'}},
    {'location': {'evidences': [{'evidenceCode': 'ECO:0000250',
        'source': 'UniProtKB',
        'id': 'D3YXG0'}],
      'value': 'Cleavage furrow',
      'id': 'SL-0467'}}]},
  {'commentType': 'ALTERNATIVE PRODUCTS',
   'events': ['Alternative splicing'],
   'isoforms': [{'name': {'value': '1'},
     'isoformIds': ['Q96RW7-1'],
     'isoformSequenceStatus': 'Displayed'},
    {'name': {'value': '2'},
     'isoformIds': ['Q96RW7-2'],
     'sequenceIds': ['VSP_016874'],
     'isoformSequenceStatus': 'Described'},
    {'name': {'value': '3'},
     'isoformIds': ['Q96RW7-3'],
     'sequenceIds': ['VSP_016871', 'VSP_016872', 'VSP_016873'],
     'isoformSequenceS

You can also request multiple records:

In [6]:
printer._depth = 2

In [7]:
Uniprotkb.fetch_many(["A0A0C5B5G6", "A0A1B0GTW7"])

[{'entryType': 'UniProtKB reviewed (Swiss-Prot)',
  'primaryAccession': 'A0A0C5B5G6',
  'uniProtkbId': 'MOTSC_HUMAN',
  'entryAudit': {'firstPublicDate': '2016-04-13',
   'lastAnnotationUpdateDate': '2022-05-25',
   'lastSequenceUpdateDate': '2015-04-29',
   'entryVersion': 22,
   'sequenceVersion': 1},
  'annotationScore': 5.0,
  'organism': {'scientificName': 'Homo sapiens',
   'commonName': 'Human',
   'taxonId': 9606,
   'evidences': [{'evidenceCode': 'ECO:0000312',
     'source': 'EMBL',
     'id': 'AJM13597.1'}],
   'lineage': ['Eukaryota',
    'Metazoa',
    'Chordata',
    'Craniata',
    'Vertebrata',
    'Euteleostomi',
    'Mammalia',
    'Eutheria',
    'Euarchontoglires',
    'Primates',
    'Haplorrhini',
    'Catarrhini',
    'Hominidae',
    'Homo']},
  'proteinExistence': '1: Evidence at protein level',
  'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000303',
       'source': 'PubMed',
       'id': '25738459'}],
     'value': 'Mitochondrial-derived peptide MOTS-c'}},
   'alternativeNames': [{'fullName': {'evidences': [{'evidenceCode': 'ECO:0000303',
        'source': 'PubMed',
        'id': '25738459'}],
      'value': 'Mitochondrial open reading frame of the 12S rRNA-c'}}]},
  'genes': [{'geneName': {'evidences': [{'evidenceCode': 'ECO:0000312',
       'source': 'HGNC',
       'id': 'HGNC:7470'}],
     'value': 'MT-RNR1'}}],
  'comments': [{'texts': [{'evidences': [{'evidenceCode': 'ECO:0000269',
        'source': 'PubMed',
        'id': '25738459'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '29886458'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '29983246'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '30468456'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '31081069'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '33468709'},
       {'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '33554779'}],
      'value': "Regulates insulin sensitivity and metabolic homeostasis (PubMed:25738459, PubMed:33468709). Inhibits the folate cycle, thereby reducing de novo purine biosynthesis which leads to the accumulation of the de novo purine synthesis intermediate 5-aminoimidazole-4-carboxamide (AICAR) and the activation of the metabolic regulator 5'-AMP-activated protein kinase (AMPK) (PubMed:25738459). Protects against age-dependent and diet-induced insulin resistance as well as diet-induced obesity (PubMed:25738459). In response to metabolic stress, translocates to the nucleus where it binds to antioxidant response elements (ARE) present in the promoter regions of a number of genes and plays a role in regulating nuclear gene expression in an NFE2L2-dependent manner and increasing cellular resistance to metabolic stress (PubMed:29983246). Increases mitochondrial respiration and levels of CPT1A and cytokines IL1B, IL6, IL8, IL10 and TNF in senescent cells (PubMed:29886458). Increases activity of the serine/threonine protein kinase complex mTORC2 and reduces activity of the PTEN phosphatase, thus promoting phosphorylation of AKT (PubMed:33554779). This promotes AKT-mediated phosphorylation of transcription factor FOXO1 which reduces FOXO1 activity, leading to reduced levels of MSTN and promotion of skeletal muscle growth (PubMed:33554779). Promotes osteogenic differentiation of bone marrow mesenchymal stem cells via the TGFB/SMAD pathway (PubMed:30468456). Promotes osteoblast proliferation and osteoblast synthesis of type I collagens COL1A1 and COL1A2 via the TGFB/SMAD pathway (PubMed:31081069)"}],
    'commentType': 'FUNCTION'},
   {'texts': [{'evidences': [{'evidenceCode': 'ECO:0000269',
        'source': 'PubMed',
        'id': '29983246'}],
      'value': 'Interacts with transcription factors ATF1 and NFE2L2/NRF2; the interactions occur in the nucleus following metabolic stress (PubMed:29983246). Also interacts with transcription factor NFE2L1/NRF1 (PubMed:2998

### ID Mapping

Unipressed also provides one other unique client, which is designed for mapping identifiers. You provide the source and destination database (both of which will autocomplete in VS Code), and a list of identifiers for the source database.

In [8]:
from unipressed import IdMappingRequest
request = IdMappingRequest(
    source="UniProtKB_AC-ID", dest="Gene_Name", ids={"A1L190", "A0JP26", "A0PK11"}
).submit()
list(request.each_result())

[{'from': 'A0PK11', 'to': 'CLRN2'},
 {'from': 'A0JP26', 'to': 'POTEB3'},
 {'from': 'A1L190', 'to': 'SYCE3'}]

Note that, if you submit a large number of IDs, you might need to add a `sleep()` call between submitting the request and retrieving the results:

### Query Syntax

You can't go wrong by following the type hints.
I strongly recommend using something like [`pylance`](https://marketplace.visualstudio.com/items?itemName=ms-python.vscode-pylance) for [Visual Studio Code](https://code.visualstudio.com/), which will provide automatic completions and warn you when you have used the wrong syntax.

If you already know how to use the Uniprot query language, you can always just input your queries as strings:
```python
UniprotkbSearch(query="(gene:BRCA*) AND (organism_id:10090)")
```


However, if you want some built-in query validation and code completion using Python's type system, then you can instead use a dictionary.
The simplest query is a dictionary with a single key: 
```python
UniprotkbSearch(query={ "family": "kinase"})
```
For brevity, for the rest of this section we will omit everything but the value of the `query` argument.

You can compile more complex queries using the `and_`, `or_` and `not_` keys.
These first two operators take a list of query dictionaries: 
```python
{
    "and_": [
        {"family": "kinase"},
        {"organism_id": "9606"},
    ]
}
```

Most "leaf" nodes of the query tree (ie those that aren't operators like `and_`) are strings, integers or floats, which you input as normal Python literals as you can see above.
For string fields, you also have access to wildcards, namely the `*` character. 
For example, if you want every human protein belonging to a gene whose name starts with `PRO`, you could use:
```python
{
    "and_": [
        {"gene": "PRO*"},
        {"organism_id": "9606"},
    ]
}
```

A few query fields are *ranges*, which you input using a tuple with two elements, indicating the start and end of the range.
If you use the literal `"*"` then you can leave the range open at one end. 
For example, this query returns any protein that is in the range $(5000, \infty)$
```python
{"length": (5000, "*")}
```

Finally, a few query fields take dates.
These you input as a Python `datetime.date` object.
For example, to find proteins added to UniProt since July 2022, we would do:
```python
from datetime import date

UniprotkbSearch(query={"date_created": (date(2022, 7, 1), "*")})
```

### Use with Visual Studio Code
To get VS Code to offer suggestions, press the `Trigger Suggest` shortcut which is usually bound to `Ctrl + Space`.
In particular, code completion generally won't work *until* you open a string literal using a quotation mark.

Secondly, to get live access to the documentation, you can either use the `Show Hover` shortcut, which is usually bound to `Ctrl + K, Ctrl + I`, or you can install the [`docs-view`](https://marketplace.visualstudio.com/items?itemName=bierner.docs-view) extension, which lets you view the docstrings in the sidebar without interfering with your code.